In [1]:
import os
import cv2
import time
import numpy as np
import pandas as pd
import mediapipe as mp
from matplotlib import pyplot as plt

In [2]:
# Grabbing the Holistic Model from Mediapipe and
# Initializing the Model

mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
	min_detection_confidence=0.2,
	min_tracking_confidence=0.2
)

# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils


In [4]:
video_path = "G:/Train/"
video_folder = os.listdir(video_path)

X = []
y = []

for video in video_folder:
    filename = video.split(".")[0]
    print(filename) 

    video_dir = video_path + video
    capture = cv2.VideoCapture(video_dir)
    previousTime = 0
    currentTime = 0

    curr_landmarks = []    

    while capture.isOpened():
	        # capture frame by frame
            ret, frame = capture.read()
            if not ret:
                break

	        # resizing the frame for better view
            frame = cv2.resize(frame, (800, 600))
            
            # Converting the from BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

	        # Making predictions using holistic model
	        # To improve performance, optionally mark the image as not writeable to
	        # pass by reference.
            
            image.flags.writeable = False
            results = holistic_model.process(image)
            image.flags.writeable = True

	        # Converting back the RGB image to BGR
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
			# Process pose
            pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
            face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            frame_pose = np.concatenate([pose, face, lh, rh])
            curr_landmarks.append(frame_pose)
            
            # frame_pose = pose+face+lh+rh
            # frame_pose = np.concatenate([pose, face, lh, rh])
            # curr_landmarks.append(frame_pose)       
            # image = np.zeros((512,512,3), np.float)

	        # Drawing the Facial Landmarks
            mp_drawing.draw_landmarks(
	            image,
	            results.face_landmarks,
	            mp_holistic.FACEMESH_CONTOURS,
	            mp_drawing.DrawingSpec(
		        color=(255,0,255),
		        thickness=1,
		        circle_radius=0
	        ),
	
            mp_drawing.DrawingSpec(
		        color=(0,255,255),
		        thickness=1,
		        circle_radius=0
	        )
	        )

	        # Drawing Right hand Land Marks
            mp_drawing.draw_landmarks(
	            image,
	            results.right_hand_landmarks,
	            mp_holistic.HAND_CONNECTIONS
	        )

	        # Drawing Left hand Land Marks
            mp_drawing.draw_landmarks(
	            image,
	            results.left_hand_landmarks,
	            mp_holistic.HAND_CONNECTIONS
	        )

	        # Drawing pose Land Marks
            mp_drawing.draw_landmarks(
	            image,
	            results.pose_landmarks,
	            mp_holistic.POSE_CONNECTIONS
	        )
	
	        # Calculating the FPS
            currentTime = time.time()
            fps = 1 / (currentTime-previousTime)
            previousTime = currentTime
	
	        # Displaying FPS on the image
            cv2.putText(image, str(int(fps))+" FPS", (10, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

	        # Display the resulting image
            cv2.imshow("Facial and Hand Landmarks", image)

	        # Enter key 'q' to break the loop
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

            # When all the process is done
            # Release the capture and destroy all windows
    
    y.append(filename)
    X.append(curr_landmarks)
        
    capture.release()
    cv2.destroyAllWindows()
print(len(X))
    

Miscarriage
Breast
Breastfeed
Health
Labor
Medicine
Cold
Diarrhea
Headache
Malaria
Tuberclosis
STI
Ebola
Cancer
HIV
AIDS
Transmit
Nurse
Doctor
Blood
I am
Have
22


In [29]:
new_x = []
for marks in X:
    new_points = []
    for points in marks:
        
        new_points = np.concatenate([np.array(new_points), points])
    new_x.append(new_points)


In [47]:
new_x = np.array(new_x)

In [21]:
X

[[array([ 0.49754351,  0.24339849, -1.08699799, ...,  0.56387973,
          0.81214786, -0.06409286]),
  array([ 0.50140274,  0.22510312, -1.14679539, ...,  0.56702536,
          0.7974847 , -0.05792229]),
  array([ 0.50218242,  0.22310859, -1.09232247, ...,  0.58099371,
          0.78162855, -0.06402194]),
  array([ 0.50289309,  0.22206062, -1.04607236, ...,  0.58566338,
          0.765661  , -0.05230395]),
  array([ 0.5056904 ,  0.22306211, -1.03055346, ...,  0.59331363,
          0.75803286, -0.04416928]),
  array([ 0.50634187,  0.22382456, -1.02342212, ...,  0.59573269,
          0.75176668, -0.05096801]),
  array([ 0.50659418,  0.22351061, -1.03173161, ...,  0.5977847 ,
          0.74525541, -0.04650658]),
  array([ 0.50776273,  0.22376004, -1.01507246, ...,  0.59961247,
          0.7428661 , -0.0409477 ]),
  array([ 0.50814116,  0.22205846, -1.04543638, ...,  0.60100031,
          0.74513137, -0.03930411]),
  array([ 0.50881273,  0.22143379, -1.02986002, ...,  0.60081518,
       